## The Line below should exists as a cell by itself

In [ ]:
%%configure -f
{ "jars": ["wasbs://jars@tcbstoragecdm.blob.core.windows.net/spark-cdm-connector-assembly-0.3.jar"]}

In [ ]:
val appid="c33b62ef-558c-4046-ac45-6a7c3a9a4745"
val appkey="@=D:8H9JFqn@SmAuwn3eZwy6bjAc90d1"
val tenantid="72f988bf-86f1-41af-91ab-2d7cd011db47"


val storage = "tcbstoragecdm.dfs.core.windows.net"
val outputContainer="/output"
val entity = "TestEntity"

## Simple Test to demonstrate parquet data writes and read with CDM metadata

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql._

val data = Seq(
    Row("tim", 1),
    Row("brad", 10)
)

val schema = (new StructType()
    .add(StructField("name", StringType, true))
    .add(StructField("id", IntegerType, true)))

val df = spark.createDataFrame(spark.sparkContext.parallelize(data),schema)

//Create a new manifest and add the entity to it
(df.write.format("com.microsoft.cdm")
    .option("storage", storage)
    .option("container", outputContainer)
    .option("manifest", "default.manifest.cdm.json")
    .option("entity", entity)
    .option("format", "parquet")
    .option("appId", appid)
    .option("appKey", appkey)
    .option("tenantId", tenantid)
    .save())

val partitions = df.rdd.getNumPartitions

val readDf = (spark.read.format("com.microsoft.cdm")
    .option("storage", storage)
    .option("container", outputContainer)
    .option("manifest", "default.manifest.cdm.json")
    .option("entity", entity)
    .option("appId", appid)
    .option("appKey", appkey)
    .option("tenantId", tenantid)
    .load())

df.select("*").show(2)
val res = df.select("name").collect()(0)
assert(df.select("name").collect()(0).getString(0) == "tim")
assert(df.select("id").collect()(1).getInt(0) == 10)


## Clean-up the contents from ADLS

In [ ]:
import com.microsoft.cdm.utils.{AADProvider,ADLGen2Provider}
import java.io.StringWriter
import org.apache.commons.io.IOUtils

val adlProvider = new ADLGen2Provider(new AADProvider(appid, appkey, tenantid))
adlProvider.deleteFile("https://" + storage + outputContainer + "/" + entity +".cdm.json")
adlProvider.deleteFile("https://" + storage + outputContainer + "/default.manifest.cdm.json")
adlProvider.deleteFile("https://" + storage + outputContainer + "/config.json")
for (i <-0 until partitions) {
    adlProvider.deleteFile("https://" + storage + outputContainer + "/" + entity + "/pdata" + i + ".parquet")
}

